In [ ]:
!pip install tensorflow
!pip3 install --upgrade tensorflow-gpu --user


In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import gensim
from gensim.test.utils import datapath
from gensim.models import KeyedVectors
import gensim.downloader as api
from sklearn.utils import class_weight
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
from keras.preprocessing.text import Tokenizer
text_train_des, text_test_des, y_train_des, y_test_des = train_test_split(df['description'],df['seed_success'], test_size=0.05, random_state=2)
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Activation,Dropout
from sklearn.metrics import accuracy_score
import tensorflow as tf


In [ ]:
df = pd.read_csv('final_seed2.csv', engine='python')

In [ ]:

df.replace(np.nan, 0, inplace=True)
df.replace('NaN', 0, inplace=True)
df.replace(' ', 0, inplace=True)

df = df.fillna(0)

In [ ]:
class_weight_d = class_weight.compute_class_weight('balanced', np.unique(y_train_des), y_train_des)
class_weight_d = dict(enumerate(class_weight_d))
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
tokenizer.fit_on_texts(text_train_des)
X_train_des = tokenizer.texts_to_sequences(text_train_des)
X_test_des = tokenizer.texts_to_sequences(text_test_des)


In [ ]:
from keras_preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Embedding
from keras.models import load_model
from keras.layers import InputLayer, Conv1D, Dense, Flatten, MaxPooling1D,Input,Concatenate
from keras import Model
def cnn_classifier(tokenizer):
    word_index = tokenizer.word_index
    vocab_size = len(word_index) + 1  # Adding 1 because of reserved 0 index

    maxlen = 100 # or fixed length for improved efficiency
    embedding_dim = 300
    print('vocabubary size:',vocab_size)
    print('max length text:',maxlen)
 
    X_train_des = pad_sequences(X_train_des, padding='post', maxlen=maxlen)
    X_test_des = pad_sequences(X_test_des, padding='post', maxlen=maxlen)


    vocab_size = len(word_index) + 1
    embedding_weights = np.zeros((len(word_index)+1, embedding_dim))
    for word,index in word_index.items():
         embedding_weights[index,:] = word2vec[word] if word in word2vec else np.random.rand(embedding_dim)
    print(embedding_weights.shape)
    nonzero_elements = np.count_nonzero(np.count_nonzero(embedding_weights, axis=1))
    print(nonzero_elements / vocab_size) 

# Deep CNN
    model = Sequential()
    model.add(Embedding(vocab_size, embedding_dim, input_length=maxlen,weights=[embedding_weights],trainable=False))
    model.add(Conv1D(128, 7, activation='relu',padding='same'))
    model.add(MaxPooling1D())
    model.add(Conv1D(256, 5, activation='relu',padding='same'))
    model.add(MaxPooling1D())
    model.add(Conv1D(512, 3, activation='relu',padding='same'))
    model.add(MaxPooling1D())
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
    model.summary()

    sequence_input = Input(shape=(maxlen,), dtype='int32')

    embedding_layer = Embedding(vocab_size,
                            embedding_dim,
                            weights=[embedding_weights],
                            input_length=maxlen,
                            trainable=False)

    embedded_sequences = embedding_layer(sequence_input)
    convs = []
    filter_sizes = [3,5,7]

    for fsz in filter_sizes:
        x = Conv1D(128, fsz, activation='relu',padding='same')(embedded_sequences)
        x = MaxPooling1D()(x)
        convs.append(x)
    
    x = Concatenate(axis=-1)(convs)
    x = Flatten()(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.5)(x)
    output = Dense(1, activation='sigmoid')(x)

    model = Model(sequence_input, output)
    model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# Fit model
    history = model.fit(X_train_des, y_train_des,
                    class_weight = class_weights_d,
                    epochs=3,
                    verbose=True,
                    validation_data=(X_test_des, y_test_des),
                    batch_size=50)
    loss, accuracy = model.evaluate(X_train_des, y_train_des, verbose=True)
    return  history